In [1140]:
import pickle
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.model_selection import train_test_split

In [1141]:
with open('train_all_22.txt','rb') as file:
    dataframe_train=pickle.load(file)
train, test = train_test_split(dataframe_train, test_size=0.2, random_state=1)
scaler = StandardScaler()

In [1142]:
#dataframe_train.describe()

In [1143]:
k_= 22 # control the number of features
#dataframe_train.columns[1:]

In [1144]:
#'''
import seaborn as sns
import matplotlib.pyplot as plt
import sys

LI=[]
for name in dataframe_train.columns[1:]:
        fig, ax = plt.subplots(figsize=(5, 4))
        for group in [0,1]:
            try:
                sns.distplot(dataframe_train.loc[dataframe_train.Label == group, name],kde=True, label=group)
            except:
                LI.append(name)  
        ax.set_ylabel('Total Count')
        ax.set_title(name)
        ax.legend()
#'''

"\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport sys\n\nLI=[]\nfor name in dataframe_train.columns[1:]:\n        fig, ax = plt.subplots(figsize=(5, 4))\n        for group in [0,1]:\n            try:\n                sns.distplot(dataframe_train.loc[dataframe_train.Label == group, name],kde=True, label=group)\n            except:\n                LI.append(name)  \n        ax.set_ylabel('Total Count')\n        ax.set_title(name)\n        ax.legend()\n"

In [1145]:
# Train set
X_train = train.drop('Label',axis=1)
y_train = train.Label

# est set
X_test = test.drop('Label',axis=1)
y_test = test.Label

## Feature Selection

statictial selection
https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/

In [1146]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif

In [1147]:
fs = SelectKBest(score_func=f_classif, k=k_)
fs.fit(X_train,y_train)
'''
# the score for all features
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))
'''

"\n# the score for all features\nfor i in range(len(fs.scores_)):\n    print('Feature %d: %f' % (i, fs.scores_[i]))\n"

In [1148]:
x_train = fs.transform(X_train)

x_test = fs.transform(X_test)

In [1149]:
#x_train.shape

## 1. Naive Bayes

In [1150]:
from sklearn.naive_bayes import GaussianNB

In [1151]:
gnb_model = GaussianNB()
gnb_model.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [1152]:
labelnb = gnb_model.predict(x_test)
probnb = gnb_model.predict_proba(x_test)

In [1153]:
nb=roc_auc_score(y_test,labelnb)
ac1=precision_score(y_test,labelnb)
print('The ROC of Naive Bayes: %5.3f' %(nb))

The ROC of Naive Bayes: 0.998


## 2. Logistic Regression

In [1154]:
from sklearn.linear_model import LogisticRegression

In [1155]:
LR_model = LogisticRegression(penalty ='none',max_iter=500)
LR_model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [1156]:
label_predict=LR_model.predict(x_test)
prob_preidct=LR_model.predict_proba(x_test)

In [1157]:
rl=roc_auc_score(y_test,label_predict)
ac2=precision_score(y_test,label_predict)
print('The ROC of Logistic Regression: %5.3f' %(rl))

The ROC of Logistic Regression: 0.528


## 3. Decision Tree and Random Forest

In [1158]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [1159]:
# Decision Tree
dt_model=DecisionTreeClassifier()

# Random Forest
rf_model=RandomForestClassifier()

In [1160]:
# Decision Tree
dt_model.fit(x_train, y_train)

# Random Forest
rf_model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [1161]:
# Decision Tree
labeldt=dt_model.predict(x_test)
probdt = dt_model.predict_proba(x_test)

# Random Forest
labelrf = rf_model.predict(x_test)
probrf = rf_model.predict_proba(x_test)

In [1162]:
dt=roc_auc_score(y_test,labeldt)
ac3=precision_score(y_test,labeldt)
print('The ROC of Decision Tree: %5.3f' %(dt))
rf=roc_auc_score(y_test,labelrf)
ac4=precision_score(y_test,labelrf)
print('The ROC of Random Forest: %5.3f' %(rf))

The ROC of Decision Tree: 1.000
The ROC of Random Forest: 1.000


## 4. KNN

In [1163]:
from sklearn.neighbors import KNeighborsClassifier

In [1164]:
knn = KNeighborsClassifier()

In [1165]:
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [1166]:
labelknn = knn.predict(x_test)
probrknn = knn.predict_proba(x_test)

In [1167]:
kn=roc_auc_score(y_test,labelknn)
ac5=precision_score(y_test,labelknn)
print('The ROC of KNN: %5.3f' %(kn))

The ROC of KNN: 1.000


## 5. SGDClassifier with Loss='log' 

In [1168]:
from sklearn.linear_model import SGDClassifier

In [1169]:
x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

In [1170]:
SGDcl=SGDClassifier(loss='log',max_iter=1000, tol=1e-3,fit_intercept=False)  
SGDcl.fit(x_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=False,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [1171]:
label_predictC=SGDcl.predict(x_test)
prob_preidctC=SGDcl.predict_proba(x_test)

In [1172]:
cl=roc_auc_score(y_test,label_predictC)
ac6=precision_score(y_test,label_predictC)
print('The ROC of SGDClassifier: %5.3f' %(cl))

The ROC of SGDClassifier: 1.000


## 6. SVM 
with RFE feature selection: recusive feature selection (model_selection)

In [1173]:
from sklearn.svm import SVC,SVR
from sklearn.feature_selection import RFE
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
estimator = SVR(kernel="linear")

In [1174]:
#RFE_model = RFE(estimator, n_features_to_select=20, step=1)

In [1175]:
#x_train=selector.transform(X_train)
SVM_model = make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True))
SVM_model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=True,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [1176]:
labelsvm = SVM_model.predict(x_test)
probrsvm = SVM_model.predict_proba(x_test)

In [1177]:
svm=roc_auc_score(y_test,labelsvm)
ac7=precision_score(y_test,labelsvm)
print('The ROC of SVM: %5.3f' %(svm))

The ROC of SVM: 1.000


## 7. MultiLayer Perceptron

In [1178]:
from sklearn.neural_network import MLPClassifier

In [1179]:
mlp_model = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10),activation='logistic', random_state=1,max_iter=3000)

In [1180]:
mlp_model.fit(x_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=10, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=3000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [1181]:
probmlp=mlp_model.predict_proba(x_test)
labelmlp=mlp_model.predict(x_test)

In [1182]:
mlp=roc_auc_score(y_test,labelmlp)
ac8=precision_score(y_test,labelmlp)
print('The ROC of MLP: %5.3f' %(mlp))

The ROC of MLP: 1.000


## 8. RFE Model

## Show Results

In [1183]:
score = [nb,rl,dt,rf,kn,cl,svm,mlp]#,rfe
precision=[ac1,ac2,ac3,ac4,ac5,ac6,ac7,ac8]#ac9
index1= ['Naive Bayes','Logistic Regression','Decision Tree','Random Forest','K-Nearest Neighbor','SGDClassifier','SVM','Multilayer Perceptron']#'Recursive Feature Elimination'
result = pd.DataFrame(score,index=index1, columns=['ROC'])
result['Precision'] = precision
#result

### Label the Given DataSet with Probability

Untill now **Random Forest** gives the higtest ROC score 0.7622

In [1184]:
import pandas as pd

In [1185]:
with open('Test_22.txt','rb') as file:
    test=pickle.load(file)

In [1186]:
Test = fs.transform(test)
Test.shape

(2000, 1)

In [1187]:
with open('ID.txt','rb') as file:
    ID=pickle.load(file)

In [1188]:
def submission(probs, filename):
    with open(filename, 'w') as file:
        file.write('Id,Predicted\n')
        for i, p in zip(ID, probs):
            file.write("{},{}\n".format(i, p[1]))

In [1189]:
def submission1(probs, filename):
    with open(filename, 'w') as file:
        file.write('Id,Predicted\n')
        for i, p in zip(ID, probs):
            file.write("{},{}\n".format(i, p))

In [1190]:
# the models without transformed X
clf1 = [LR_model,dt_model,rf_model,knn,gnb_model]
prob1=[]
for clf in clf1:
    prob=clf.predict_proba(Test)
    prob1.append(prob)

In [1191]:
submission(prob1[0], 'LR_submit.csv')
submission(prob1[1], 'DT_submit.csv')
submission(prob1[2], 'RF_submit.csv')
submission(prob1[3], 'KNN_submit.csv')
submission(prob1[4], 'NB_submit.csv')

In [1192]:
# the models with transformed X
clf2=[SGDcl,SVM_model,mlp_model]
Test_=scaler.transform(Test)
prob2=[]
for clf in clf2:
    prob=clf.predict_proba(Test_)
    prob2.append(list(prob))

In [1193]:
submission(prob2[0], 'SGD_submit.csv')
submission(prob2[1], 'SVM_submit.csv')
submission(prob2[2], 'MLP_submit.csv')

In [1194]:
#submission1(probrfe, 'RFE_submit.csv')

In [ ]:
csvs=['NB_submit.csv','LR_submit.csv','DT_submit.csv','RF_submit.csv','KNN_submit.csv','SGD_submit.csv','SVM_submit.csv','MLP_submit.csv']#'RFE_submit.csv
P=[]
N=[]
for csv in csvs:
    dd=pd.read_csv(csv)
    l=list(dd.Predicted)
    z=[]
    o=[]
    for elem in l:
        if elem <=0.5:
            z.append(elem)
        else:
            o.append(elem)
    P.append(len(o))
    N.append(len(z))
result['Positive'] = P
result['Negative'] = N
result